Projeto desenvolvido por Andreza Lima no âmbito do módulo de Extração de Dados I, na trilha de Engenharia de dados do programa Santander Coders 2023, em parceria com a Ada Tech.

Desenvolvido com Databricks Community edition.

# Projeto Sistema de Monitoramento de Avanços no Campo da Genômica

Contexto:

O grupo trabalha no time de engenharia de dados na HealthGen, uma empresa especializada em genômica e pesquisa de medicina personalizada. A genômica é o estudo do conjunto completo de genes de um organismo, desempenha um papel fundamental na medicina personalizada e na pesquisa biomédica. Permite a análise do DNA para identificar variantes genéticas e mutações associadas a doenças e facilita a personalização de tratamentos com base nas características genéticas individuais dos pacientes.

A empresa precisa se manter atualizada sobre os avanços mais recentes na genômica, identificar oportunidades para pesquisa e desenvolvimento de tratamentos personalizados e acompanhar as tendências em genômica que podem influenciar estratégias de pesquisa e desenvolvimento. Pensando nisso, o time de dados apresentou uma proposta de desenvolvimento de um sistema que coleta, analisa e apresenta as últimas notícias relacionadas à genômica e à medicina personalizada, e também estuda o avanço do campo nos últimos anos. 

O time de engenharia de dados tem como objetivo desenvolver e garantir um pipeline de dados confiável e estável. As principais atividades são:

1. Consumo de dados com a News API: 
Implementar um mecanismo para consumir dados de notícias de fontes confiáveis e especializadas em genômica e medicina personalizada, a partir da News API: 
https://newsapi.org/

2. Definir Critérios de Relevância:
Desenvolver critérios precisos de relevância para filtrar as notícias. Por exemplo, o time pode se concentrar em notícias que mencionem avanços em sequenciamento de DNA, terapias genéticas personalizadas ou descobertas relacionadas a doenças genéticas específicas.

3. Cargas em Batches:
Armazenar as notícias relevantes em um formato estruturado e facilmente acessível para consultas e análises posteriores. Essa carga deve acontecer 1 vez por hora. Se as notícias extraídas já tiverem sidos armazenadas na carga anterior, o processo deve ignorar e não armazenar as notícias novamente, os dados carregados não podem ficar duplicados.

<br>

<div style="text-align: center;">
<img src="https://drive.google.com/uc?export=view&id=1QLZBxgK4c4_yysUnvtamuwXzRJm4nNit"  width="70%" height="40%">
<br>
<br>

</div>

4. Dados transformados para consulta do público final:
A partir dos dados carregados, aplicar as seguintes transformações e armazenar o resultado final para a consulta do público final:

    4.1 - Quantidade de notícias por ano, mês e dia de publicação;

    4.2 - Quantidade de notícias por fonte e autor;
    
    4.3 - Quantidade de aparições de 3 palavras chaves por ano, mês e dia de publicação (as 3 palavras chaves serão as mesmas usadas para fazer os filtros de relevância do item 2 (2. Definir Critérios de Relevância)).

Atualizar os dados transformados 1 vez por dia.

<br>

<div style="text-align: center;">
<img src="https://drive.google.com/uc?export=view&id=1QOFkzKrWqb-9CY3kC3_1XkTWNVNE05dd"  width="70%" height="40%">
<br>
<br>

</div>

Além das atividades principais, existe a necessidade de busca de dados por eventos em tempo real quando é necessário, para isso foi desenhado duas opções:

Opção 1 - Apache Kafka e Spark Streaming:

Preparar um pipeline com Apache Kafka e Spark Streaming para receber os dados do Produtor Kafka representado por um evento manual e consumir os dados com o Spark Streaming armazenando os resultados temporariamente. Em um processo paralelo, verificar os resultados armazenados temporiamente e armazenar no mesmo destino do item 3 (3. Cargas em Batches) aqueles resultados que ainda não foram armazenados no destino (os dados carregados não podem ficar duplicados). E por fim, eliminar os dados temporários após a verificação e a eventual carga.

<br>

<div style="text-align: center;">
<img src="https://drive.google.com/uc?export=view&id=1PvAxBXU0fvwEtJg36ZJ1VfBVSGETBpUZ"  width="70%" height="40%">
<br>
<br>

</div>


Opção 2 - Webhooks com notificações por eventos:

Configurar um webhook para adquirir as últimas notícias a partir de um evento representado por uma requisição POST e fazer a chamada da API e por fim armazenar os resultados temporariamente. Em um processo paralelo, verificar os resultados armazenados temporiamente e armazenar no mesmo destino do item 3 (3. Cargas em Batches) aqueles resultados que ainda não foram armazenados no destino (os dados carregados não podem ficar duplicados). E por fim, eliminar os dados temporários após a verificação e a eventual carga.

<br>

<div style="text-align: center;">
<img src="https://drive.google.com/uc?export=view&id=1Px6Jp3aNuF-wpn_9earonylEMebzOcBW"  width="70%" height="40%">
<br>
<br>

</div>

## Atividades que precisam ser realizadas pelo grupo:

O grupo precisa construir o pipeline de dados seguindo os requisitos das atividades principais e escolher entre a Opção 1 e Opção 2 para desenvolvimento.

# Proposta de Resolução

A execução deste projeto foi dividida em 5 arquivos.

1-ada-projeto-load-data: contém os passos necessários para carregamento dos dados de hora em hora

2-ada-projeto-transform-data (este arquivo): contém os passos necessários para transformação do arquivo consolidado, a cada 24 horas

3-ada-projeto-webhook: cria uma aplicação flask para o processo de webhook

4-ada-projeto-load-temp-data: contém os passos necessários para carregamento dos dados em uma pasta temporária, e depois carregamento no arquivo principal

5-ada-projeto-webhook-post: envia post commands para a aplicação criada no arquivo 3-ada-projeto-webhook

## Importando Pacotes

In [0]:
## Executar apenas se for necessário instalar pacotes
# !pip install pyspark
# !pip install flask

In [0]:
from datetime import datetime
# import flask
import json
import pyspark.pandas as ps
import requests
import time

## 1. Consumo de dados com a News API

Ir para o arquivo 1-ada-projeto-load-data

## 2. Definir Critérios de Relevância

De acordo com Antunes (2022)¹, após cerca de 20 anos depois da publicação do primeiro rascunho da sequência de todo o genoma humano, os investigadores da área mergulharam na era “pós-genómica”, remodelando a investigação biológica e abrindo portas para a chamada medicina personalizada ou medicina de precisão, que pesquisa como medicamentos podem ser adaptados de um modo preciso, proporcionando o melhor tratamento possível através da sequenciação do genoma de cada indivíduo.

Tendo como base essa discussão sobre os avanços da genômica, as palavras-chave escolhidas para este projeto foram os termos em inglês *genomics*, *precision medicine* e *personalized medicine*.

¹ Antunes, A., (2022) Avanços da genómica, Rev. Ciência Elem., V10(4):056. Disponível em: https://rce.casadasciencias.org/rceapp/art/2022/056/. Acesso em: 25 set. 2023.

## 3. Cargas em Batches:

Ir para o arquivo 1-ada-projeto-load-data

## 4. Dados transformados para consulta do público final:
A partir dos dados carregados, aplicar as seguintes transformações e armazenar o resultado final para a consulta do público final:

    4.1 - Quantidade de notícias por ano, mês e dia de publicação;

    4.2 - Quantidade de notícias por fonte e autor;
    
    4.3 - Quantidade de aparições de 3 palavras chaves por ano, mês e dia de publicação (as 3 palavras chaves serão as mesmas usadas para fazer os filtros de relevância do item 2 (2. Definir Critérios de Relevância)).

Atualizar os dados transformados 1 vez por dia.

In [0]:
# Função com a transformação dos dados
def transform_data(dir_escrita_bruto, dir_escrita_transformado):
    # Leitura dos dados carregados
    print("\nIniciando leitura do arquivo consolidado.")
    df = ps.read_parquet(dir_escrita_bruto) 


    ##### 4.1. Quantidade de notícias por ano, mês e dia de publicação #####
    print("\n> Efetuando de Transformações")
    # Cria um dataframe a partir do df original
    # Agrega por publishedAt_ano, publishedAt_mes, publishedAt_dia
    # Conta as urls
    df1 = df.groupby(["publishedAt_ano","publishedAt_mes","publishedAt_dia"]).agg(count_news=('url', 'count'))
    df1.reset_index(inplace=True)

    ##### 4.2 - Quantidade de notícias por fonte e autor #####
    # Cria um dataframe a partir do df original
    # Agrega por source_name, author
    # Conta as urls
    df2 = df.groupby(["source_name","author"]).agg(count_news=('url', 'count'))
    df2.reset_index(inplace=True)
    
    ##### 4.3 - Quantidade de aparições de 3 palavras chaves por ano, mês e dia de publicação (as 3 palavras chaves serão as mesmas usadas para fazer os filtros de relevância do item 2 (2. Definir Critérios de Relevância)) #####
    # Cria uma coluna com a contagem de keywords
    # Cria um dataframe a partir do df original
    # Agrega por publishedAt_ano, publishedAt_mes, publishedAt_dia
    # Soma as palavras chave
    df["keywords"] = df["content"].str.count("brasil") + df["content"].str.count("as")
    df3 = df.groupby(["publishedAt_ano","publishedAt_mes","publishedAt_dia"]).agg(count_keywords=('keywords', 'sum'))
    df3.reset_index(inplace=True)

    print("\n> Arquivos transformados com sucesso.")
    #reseta o index
    # df = df.reset_index()

    #Armazena os resultados transformados
    print("\n> Iniciando salvamento dos arquivos.")
    df1.to_parquet(dir_escrita_transformado+"dado_transformado1.parquet")
    df2.to_parquet(dir_escrita_transformado+"dado_transformado2.parquet")
    df3.to_parquet(dir_escrita_transformado+"dado_transformado3.parquet")

    print("Transformação realizada com sucesso!")

In [0]:
# Definindo variáveis para o transform_data

dir_escrita_transformado = "/FileStore/projeto/"
dir_escrita_bruto = "/FileStore/projeto/dados_consolidados.parquet"
tempo_proc_transform = 24 #* 60 # * 60 # A cada 24 horas * 60 minutos * 60 segundos

In [0]:
#loop chamando o transform_data a cada dia

contador_transform_data = 0 # Limitar para testes
while contador_transform_data < 2:
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print("\n" + timestamp+ f": Próxima verificação em {tempo_proc_transform/60} minutos.")
    time.sleep(tempo_proc_transform)
    print("\nIniciando etapa de transformação")
    transform_data(dir_escrita_bruto, dir_escrita_transformado)
    contador_transform_data +=1 # Comentar esta linha para rodar infinitamente


2023-09-29 20:57:29: Próxima verificação em 0.4 minutos.

Iniciando etapa de transformação

Iniciando leitura do arquivo consolidado.

> Efetuando de Transformações

> Arquivos transformados com sucesso.

> Iniciando salvamento dos arquivos.
Transformação realizada com sucesso!

2023-09-29 20:58:03: Próxima verificação em 0.4 minutos.


In [0]:
teste1 = ps.read_parquet("/FileStore/projeto/dado_transformado1.parquet")

In [0]:
teste1

,publishedAt_ano,publishedAt_mes,publishedAt_dia,count_news
0,2023,09,07,5
1,2023,09,27,6
2,2023,08,28,2
3,2023,09,24,1
4,2023,08,31,6
5,2023,09,20,1
6,2023,09,08,2
7,2023,09,12,2
8,2023,09,26,2
9,2023,09,15,2


In [0]:
teste2 = ps.read_parquet("/FileStore/projeto/dado_transformado2.parquet")
teste2

,source_name,author,count_news
0,Live Science,ben.turner@futurenet.com (Ben Turner),1
1,Marketscreener.com,Publicnow,1
2,GlobeNewswire,MarketsandMarkets Research Pvt. Ltd.,1
3,Marketscreener.com,Business Wire,2
4,Googleblog.com,Google Developers,1
5,Wired.co.uk,Rachael Pells,1
6,Biztoc.com,scmp.com,1
7,GlobeNewswire,XCELLENT INSIGHTS LLP,1
8,New Atlas,Bronwyn Thompson,1
9,Forbes,"Alex Knapp, Forbes Staff, \n Alex Knapp, Forbe...",1


In [0]:
teste3 = ps.read_parquet("/FileStore/projeto/dado_transformado3.parquet")
teste3

,publishedAt_ano,publishedAt_mes,publishedAt_dia,count_keywords
0,2023,09,07,2
1,2023,09,27,9
2,2023,08,28,1
3,2023,09,24,2
4,2023,08,31,5
5,2023,09,20,1
6,2023,09,08,4
7,2023,09,12,0
8,2023,09,26,1
9,2023,09,15,0


## Opção 2 - Webhooks com notificações por eventos:

Webhook é uma forma de automação de comunicação entre sistemas. É um método que permite que um sistema envie automaticamente dados ou informações para outro sistema assim que um evento específico ocorrer. 

Para fins deste projeto, será criada uma aplicação com flask para simular um evento que ocorre sempre que um post command é executado.


### Criação de uma aplicação flask para simular um evento
Configurar um webhook para adquirir as últimas notícias a partir de um evento representado por uma requisição POST e fazer a chamada da API e por fim armazenar os resultados temporariamente. Em um processo paralelo, verificar os resultados armazenados temporiamente e armazenar no mesmo destino do item 3 (3. Cargas em Batches) aqueles resultados que ainda não foram armazenados no destino (os dados carregados não podem ficar duplicados). E por fim, eliminar os dados temporários após a verificação e a eventual carga.

Ir para o arquivo 3-ada-projeto-webhook

### Carregamento de dados temporários

Ir para o arquivo 4-ada-projeto-load-temp-data

### Execução do post command

Ir para o arquivo 5-ada-projeto-webhook-post